<a href="https://colab.research.google.com/github/josephxlp/FinMLA/blob/main/Portfolio_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Allocation and Performance at Any Point Time by Random Weights

In [1]:
from google.colab import drive
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
drive.mount('/content/drive', force_remount=True)
rootpath = Path("/content/drive/MyDrive/portfolio/FinMLA")
datapath = rootpath / 'data'
os.chdir(rootpath)

Mounted at /content/drive


In [3]:
df = pd.read_csv(datapath/"stock.csv", parse_dates=['Date'])
stocks_df = df.copy()
stocks_df.sort_values("Date",inplace=True)
nrows,ncol  = stocks_df.shape

In [4]:
def gen_portfolio_random_weights(ncol,seed=None):
    if seed:
        np.random.seed(seed)
    weights = np.array(np.random.random(ncol))
    weights = weights / weights.sum()
    return weights

def normalize(df):
    x = df.copy()
    x = x.reset_index(drop=True)
    try:
        for i in x.columns:
            x[i] = x[i]/x[i][0]
    except:
        for i in x.columns[1:]:
            x[i] = x[i]/x[i][0]
    x.index = df.index
    return x

def adjust_portfolio(df_portfolio, weights, investment=1_000_000):
    """
    Adjusts the portfolio by applying weights and scaling.

    Parameters:
    - df_portfolio: DataFrame containing portfolio data. Assumes the first column is not a stock.
    - weights: List or array of weights corresponding to each stock column in df_portfolio.

    Returns:
    - DataFrame with adjusted portfolio values.
    """
    # Ensure weights is a pandas Series for easier broadcasting
    weights_series = pd.Series(weights, index=df_portfolio.columns[1:])

    # Apply weights and scale by 1,000,000
    df_portfolio.iloc[:, 1:] = df_portfolio.iloc[:, 1:].mul(weights_series, axis=1) * investment

    return df_portfolio

def estimate_portfolio_worth(data, newname, xcol='Date'):
    """
    Estimates the portfolio worth by summing all numeric columns except the specified column.

    Parameters:
    - data: DataFrame containing the portfolio data.
    - newname: The name of the new column to store the summed values.
    - xcol: The column to exclude from the sum.

    Returns:
    - DataFrame with the new column added.
    """
    if xcol in data.columns:
        # Select only numeric columns, excluding the specified column
        numeric_cols = data.select_dtypes(include='number').columns
        cols_to_sum = numeric_cols.difference([xcol])

        # Sum the selected columns
        data[newname] = data[cols_to_sum].sum(axis=1)
    else:

        data[newname] = data.sum(axis=1)

    return data


def portfolio_allocation(df, weights=None,
                        investment=1_000_000,
                        new_name='pflio_worth'):

    if weights is None:
        numeric_cols = df.select_dtypes(include='number').columns
        weights = gen_portfolio_random_weights(len(numeric_cols))
    pflio_n = normalize(df)
    pflio_n = adjust_portfolio(pflio_n, weights, investment)
    pflio_n = estimate_portfolio_worth(pflio_n,new_name)
    pflio_n['pflio_worth_ret'] = pflio_n['pflio_worth'].pct_change()
    pflio_n.dropna(inplace=True)
    return pflio_n

In [5]:
weights = gen_portfolio_random_weights(ncol-1)
pflio_n = normalize(stocks_df)
d2 = adjust_portfolio(pflio_n, weights, investment=1_000_000)
d3 = estimate_portfolio_worth(d2,'pflio_worth')
d3['pflio_worth_ret'] = d3['pflio_worth'].pct_change()
d3.dropna(inplace=True)

In [6]:
pflio_a = portfolio_allocation(stocks_df)

In [7]:
pflio_a.sample(10)

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,pflio_worth,pflio_worth_ret
503,2014-01-14,119545.540663,157129.153563,106720.519585,94833.973824,2.868507e+05,208676.670304,93423.258438,368877.061038,190771.564216,1.626828e+06,0.021355
1712,2018-10-31,335193.092251,398249.061265,97795.267051,98144.698351,1.153067e+06,129558.672076,195408.528608,693726.341286,281324.979462,3.382468e+06,0.031242
1774,2019-02-01,255032.234339,434801.435891,95627.705722,109437.960118,1.173429e+06,150513.893292,180862.367165,715618.487951,280784.480677,3.396108e+06,-0.018600
1882,2019-07-09,308207.343747,396262.653985,106911.778184,107525.094932,1.434686e+06,156384.042619,133273.107933,724689.725244,309116.773264,3.677057e+06,0.009134
1817,2019-04-04,299707.285906,444262.171230,101939.134299,99873.634191,1.312424e+06,160256.320449,155124.198119,782783.221121,298717.541320,3.655088e+06,0.001143
1493,2017-12-18,270194.482292,332349.343296,122594.715547,121209.441859,8.590800e+05,172097.647479,196306.432285,693964.716494,279086.189968,3.046883e+06,0.007788
2124,2020-06-23,561355.767186,210852.276347,96424.603269,67906.710398,1.994699e+06,134025.829947,580328.345173,943469.632475,324850.505270,4.913913e+06,0.012870
1261,2017-01-18,183769.616582,177677.948578,130053.676594,105244.374736,5.826487e+05,187216.381278,138081.276260,519323.519777,235693.458755,2.259709e+06,-0.000147
645,2014-08-07,144700.010129,134492.949125,109047.457237,90015.017635,2.247312e+05,206858.387354,146208.814528,361959.931067,198105.175209,1.616119e+06,-0.004618
2139,2020-07-15,598679.416756,210919.609239,95659.581623,66950.285162,2.171093e+06,138055.242704,895599.238109,975186.836675,334734.135898,5.486878e+06,-0.002789


In [8]:
fig = px.line(x = pflio_a.Date, y = pflio_a['pflio_worth_ret'], title = 'Portfolio Daily % Return')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [9]:
fig = px.histogram(pflio_n, x = 'pflio_worth_ret')
fig.show()

In [10]:
from vizutils import plot_timeseries, plot_timeseries_it

In [11]:
pflio_n.columns

Index(['Date', 'AAPL', 'BA', 'T', 'MGM', 'AMZN', 'IBM', 'TSLA', 'GOOG',
       'sp500', 'pflio_worth', 'pflio_worth_ret'],
      dtype='object')

In [12]:
# Plot all stocks (normalized)
plot_timeseries_it(pflio_a.drop(['pflio_worth', 'pflio_worth_ret'], axis = 1),
'Portfolio individual stocks worth in $ over time')

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [13]:

def calculate_cumulative_return(df_portfolio, column_name):
  """
  Calculates the cumulative return of the portfolio.

  Parameters:
  - df_portfolio: DataFrame containing the portfolio data.
  - column_name: The name of the column containing the portfolio's daily worth.

  Returns:
  - Cumulative return as a percentage.

  Interpretation:
  - Positive Cumulative Return: Indicates a gain in portfolio value over the period.
  - Negative Cumulative Return: Indicates a loss in portfolio value over the period.
  - Zero Cumulative Return: Indicates no change in portfolio value over the period.

  Range of Values:
  - Cumulative return can range from negative infinity (complete loss) to positive infinity (significant gain).
  """

  # Get the first and last values of the specified column
  start_value = df_portfolio[column_name].iloc[0]
  end_value = df_portfolio[column_name].iloc[-1]

  # Calculate the cumulative return
  cumulative_return = ((end_value - start_value) / start_value) * 100

  return cumulative_return

# Example usage:
# df_portfolio = pd.DataFrame(...)  # Your DataFrame with portfolio data
cumulative_return = calculate_cumulative_return(pflio_a, 'pflio_worth_ret')
print('Cumulative return of the portfolio is {:.2f} %'.format(cumulative_return))

Cumulative return of the portfolio is 231.26 %


In [14]:
# the porfolio as all is doing extremely well over the perdio in question, more than tripped the initial inversement

metrics

In [15]:
import numpy as np
import pandas as pd

def calculate_portfolio_metrics(df_portfolio, return_column):
  """
  Calculates the Sharpe ratio, standard deviation, and average daily return of the portfolio.

  Parameters:
  - df_portfolio: DataFrame containing the portfolio data.
  - return_column: The name of the column containing the portfolio's daily percentage returns.

  Returns:
  - A dictionary with the Sharpe ratio, standard deviation, and average daily return.
  """
  # Calculate the average daily return
  average_daily_return = df_portfolio[return_column].mean()

  # Calculate the standard deviation of daily returns
  std_dev = df_portfolio[return_column].std()

  # Calculate the Sharpe ratio (assuming 252 trading days in a year)
  sharpe_ratio = average_daily_return / std_dev * np.sqrt(252)

  # Print the results
  print('Sharpe ratio of the portfolio is {:.2f}'.format(sharpe_ratio))
  print('Standard deviation of the portfolio is {:.2f}'.format(std_dev))
  print('Average daily return of the portfolio is {:.2f} %'.format(average_daily_return))

  # Return the metrics as a dictionary
  return {
      'Sharpe Ratio': sharpe_ratio,
      'Standard Deviation': std_dev,
      'Average Daily Return': average_daily_return
  }

# Example usage:
# df_portfolio = pd.DataFrame(...)  # Your DataFrame with portfolio data
metrics = calculate_portfolio_metrics(pflio_a, 'pflio_worth_ret')

Sharpe ratio of the portfolio is 1.09
Standard deviation of the portfolio is 0.01
Average daily return of the portfolio is 0.00 %


The values you've provided for the Sharpe ratio, standard deviation, average daily return, and cumulative return can coexist, but they require some context to understand how they relate to each other. Let's break down each metric and how they can align:

1. **Sharpe Ratio of 1.08:**
   - The Sharpe ratio is a measure of risk-adjusted return. A ratio of 1.08 suggests that the portfolio's returns are slightly above the risk-free rate when adjusted for volatility. This is generally considered a decent Sharpe ratio, indicating that the portfolio is providing a reasonable return for the level of risk taken.

2. **Standard Deviation of 0.01:**
   - A standard deviation of 0.01 (or 1% if expressed as a percentage) indicates low volatility in the portfolio's daily returns. This means that the daily returns are relatively stable and do not fluctuate widely.

3. **Average Daily Return of 0.00%:**
   - An average daily return of 0.00% might seem low, but it can still lead to a significant cumulative return over a long period due to compounding. It's important to note that even small daily returns can accumulate to large gains over time.

4. **Cumulative Return of 348.02%:**
   - A cumulative return of 348.02% indicates that the portfolio's value has increased significantly over the entire period. This large cumulative return could be the result of consistent, albeit small, positive daily returns compounded over a long period.

### How These Metrics Can Coexist:

- **Compounding Effect:** Even if the average daily return is close to zero, if it's slightly positive and consistent over a long period, it can lead to a large cumulative return due to the power of compounding. This is especially true if the portfolio has been held for many years.

- **Low Volatility:** The low standard deviation suggests that the returns are stable, which can contribute to a higher Sharpe ratio. Investors often prefer stable returns, even if they are small, because they reduce the risk of large losses.

- **Risk-Adjusted Performance:** The Sharpe ratio of 1.08 indicates that the portfolio is providing a reasonable return for the risk taken, which aligns with the low volatility and the eventual high cumulative return.

In summary, these metrics together suggest a portfolio that has achieved significant growth over time with relatively low daily volatility and modest daily returns, resulting in a decent risk-adjusted performance.

In [16]:
def portfolio_allocation_performance(df, end_date=None):
  """
  Calculate the difference between the first row and a specified end row of a DataFrame
  and append it as a new row. Defaults to the last row if the end date is not found.

  Parameters:
  df (pd.DataFrame): The input DataFrame with a DateTime index.
  end_date (str): The end date in 'Y-M-D' format.

  Returns:
  pd.DataFrame: A DataFrame with the first row, specified end row, and their difference.
  """
  # Ensure the index is in datetime format and sort the DataFrame by date
  df.index = pd.to_datetime(df.index)
  df = df.sort_index()

  # Select the first row as the start row
  start_row = df.head(1)

  # Find the row corresponding to the end date
  if end_date:
      end_date = pd.to_datetime(end_date)
      print('end_date:', end_date)
      if end_date in df.index:
          end_row = df.loc[[end_date]]
      else:
          # Find the closest date less than or equal to the end_date
          closest_date = df.index[df.index <= end_date].max()
          if pd.isna(closest_date):
              end_row = df.tail(1)
              print(f"Date given not covered, returning last available date: {df.index[-1].date()}")
          else:
              end_row = df.loc[[closest_date]]
              print(f"Date given not covered, returning closest available date: {closest_date.date()}")
  else:
      end_row = df.tail(1)

  # Concatenate the start and end rows
  outcome = pd.concat([start_row, end_row])

  # Calculate the difference between the end and start row
  difference = outcome.iloc[1] - outcome.iloc[0]

  # Convert the difference Series to a DataFrame and concatenate
  difference_df = pd.DataFrame([difference], columns=df.columns)
  outcome = pd.concat([outcome, difference_df], ignore_index=True)

  # Add a new column for labels
  labels = ['Start Row', 'End Row', 'Difference']
  outcome['Label'] = labels

  # Set the original index back to the first two rows
  outcome.index = list(start_row.index) + list(end_row.index) + ['Difference']
  # here instead of using difference use subtraction and keep date formaation
  outcome.reset_index(drop=True, inplace=True)

  return outcome

In [17]:
portfolio_allocation_performance(pflio_a,None)

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,pflio_worth,pflio_worth_ret,Label
0,2012-01-13 00:00:00,91850.898439,83721.412472,95850.837035,45430.548150,1.287415e+05,201089.250692,13202.183298,200578.094455,133734.506091,9.941992e+05,-0.005801,Start Row
1,2020-08-11 00:00:00,670049.242392,202154.676293,96265.226947,79089.618236,2.222901e+06,142264.244006,796180.259077,953719.848247,345848.148290,5.508473e+06,-0.019216,End Row
2,3133 days 00:00:00,578198.343953,118433.263821,414.389912,33659.070087,2.094160e+06,-58825.006686,782978.075780,753141.753791,212113.642199,4.514273e+06,-0.013415,Difference


In [18]:
# next: seatch optimal weights for portfolio allocation given:
# the metrics

In [20]:
%%writefile portfolio_allocation_utils.py
# PflioA
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go


def portfolio_allocation_performance(df, end_date=None):
  """
  Calculate the difference between the first row and a specified end row of a DataFrame
  and append it as a new row. Defaults to the last row if the end date is not found.

  Parameters:
  df (pd.DataFrame): The input DataFrame with a DateTime index.
  end_date (str): The end date in 'Y-M-D' format.

  Returns:
  pd.DataFrame: A DataFrame with the first row, specified end row, and their difference.
  """
  # Ensure the index is in datetime format and sort the DataFrame by date
  df.index = pd.to_datetime(df.index)
  df = df.sort_index()

  # Select the first row as the start row
  start_row = df.head(1)

  # Find the row corresponding to the end date
  if end_date:
      end_date = pd.to_datetime(end_date)
      print('end_date:', end_date)
      if end_date in df.index:
          end_row = df.loc[[end_date]]
      else:
          # Find the closest date less than or equal to the end_date
          closest_date = df.index[df.index <= end_date].max()
          if pd.isna(closest_date):
              end_row = df.tail(1)
              print(f"Date given not covered, returning last available date: {df.index[-1].date()}")
          else:
              end_row = df.loc[[closest_date]]
              print(f"Date given not covered, returning closest available date: {closest_date.date()}")
  else:
      end_row = df.tail(1)

  # Concatenate the start and end rows
  outcome = pd.concat([start_row, end_row])

  # Calculate the difference between the end and start row
  difference = outcome.iloc[1] - outcome.iloc[0]

  # Convert the difference Series to a DataFrame and concatenate
  difference_df = pd.DataFrame([difference], columns=df.columns)
  outcome = pd.concat([outcome, difference_df], ignore_index=True)

  # Add a new column for labels
  labels = ['Start Row', 'End Row', 'Difference']
  outcome['Label'] = labels

  # Set the original index back to the first two rows
  outcome.index = list(start_row.index) + list(end_row.index) + ['Difference']
  # here instead of using difference use subtraction and keep date formaation
  outcome.reset_index(drop=True, inplace=True)

  return outcome


def calculate_portfolio_metrics(df_portfolio, return_column):
  """
  Calculates the Sharpe ratio, standard deviation, and average daily return of the portfolio.

  Parameters:
  - df_portfolio: DataFrame containing the portfolio data.
  - return_column: The name of the column containing the portfolio's daily percentage returns.

  Returns:
  - A dictionary with the Sharpe ratio, standard deviation, and average daily return.
  """
  # Calculate the average daily return
  average_daily_return = df_portfolio[return_column].mean()

  # Calculate the standard deviation of daily returns
  std_dev = df_portfolio[return_column].std()

  # Calculate the Sharpe ratio (assuming 252 trading days in a year)
  sharpe_ratio = average_daily_return / std_dev * np.sqrt(252)

  # Print the results
  print('Sharpe ratio of the portfolio is {:.2f}'.format(sharpe_ratio))
  print('Standard deviation of the portfolio is {:.2f}'.format(std_dev))
  print('Average daily return of the portfolio is {:.2f} %'.format(average_daily_return))

  # Return the metrics as a dictionary
  return {
      'Sharpe Ratio': sharpe_ratio,
      'Standard Deviation': std_dev,
      'Average Daily Return': average_daily_return
  }

def calculate_cumulative_return(df_portfolio, column_name):
  """
  Calculates the cumulative return of the portfolio.

  Parameters:
  - df_portfolio: DataFrame containing the portfolio data.
  - column_name: The name of the column containing the portfolio's daily worth.

  Returns:
  - Cumulative return as a percentage.

  Interpretation:
  - Positive Cumulative Return: Indicates a gain in portfolio value over the period.
  - Negative Cumulative Return: Indicates a loss in portfolio value over the period.
  - Zero Cumulative Return: Indicates no change in portfolio value over the period.

  Range of Values:
  - Cumulative return can range from negative infinity (complete loss) to positive infinity (significant gain).
  """

  # Get the first and last values of the specified column
  start_value = df_portfolio[column_name].iloc[0]
  end_value = df_portfolio[column_name].iloc[-1]

  # Calculate the cumulative return
  cumulative_return = ((end_value - start_value) / start_value) * 100

  return cumulative_return


def gen_portfolio_random_weights(ncol,seed=None):
    if seed:
        np.random.seed(seed)
    weights = np.array(np.random.random(ncol))
    weights = weights / weights.sum()
    return weights

def normalize(df):
    x = df.copy()
    x = x.reset_index(drop=True)
    try:
        for i in x.columns:
            x[i] = x[i]/x[i][0]
    except:
        for i in x.columns[1:]:
            x[i] = x[i]/x[i][0]
    x.index = df.index
    return x

def adjust_portfolio(df_portfolio, weights, investment=1_000_000):
    """
    Adjusts the portfolio by applying weights and scaling.

    Parameters:
    - df_portfolio: DataFrame containing portfolio data. Assumes the first column is not a stock.
    - weights: List or array of weights corresponding to each stock column in df_portfolio.

    Returns:
    - DataFrame with adjusted portfolio values.
    """
    # Ensure weights is a pandas Series for easier broadcasting
    weights_series = pd.Series(weights, index=df_portfolio.columns[1:])

    # Apply weights and scale by 1,000,000
    df_portfolio.iloc[:, 1:] = df_portfolio.iloc[:, 1:].mul(weights_series, axis=1) * investment

    return df_portfolio

def estimate_portfolio_worth(data, newname, xcol='Date'):
    """
    Estimates the portfolio worth by summing all numeric columns except the specified column.

    Parameters:
    - data: DataFrame containing the portfolio data.
    - newname: The name of the new column to store the summed values.
    - xcol: The column to exclude from the sum.

    Returns:
    - DataFrame with the new column added.
    """
    if xcol in data.columns:
        # Select only numeric columns, excluding the specified column
        numeric_cols = data.select_dtypes(include='number').columns
        cols_to_sum = numeric_cols.difference([xcol])

        # Sum the selected columns
        data[newname] = data[cols_to_sum].sum(axis=1)
    else:

        data[newname] = data.sum(axis=1)

    return data


def portfolio_allocation(df, weights=None,
                        investment=1_000_000,
                        new_name='pflio_worth'):

    if weights is None:
        numeric_cols = df.select_dtypes(include='number').columns
        weights = gen_portfolio_random_weights(len(numeric_cols))
    pflio_n = normalize(df)
    pflio_n = adjust_portfolio(pflio_n, weights, investment)
    pflio_n = estimate_portfolio_worth(pflio_n,new_name)
    pflio_n['pflio_worth_ret'] = pflio_n['pflio_worth'].pct_change()
    pflio_n.dropna(inplace=True)
    return pflio_n

Writing portfolio_allocation_utils.py
